# Build a 3D Orbital Propagator — Outreach (Starter)


Welcome! This notebook guides you from **physics** to a working **3D orbital propagator** — step by step and hands-on.
No slides needed: all the theory and code are here. It’s designed for VS Code’s Jupyter with minimal dependencies.

## How everything fits together (read this first)
You’ll build small functions that snap together into a **simulation pipeline**:

1. **Accelerations** (physics building blocks)
   - `gravity_accel(r, mu)` — point-mass gravity.
   - `j2_accel(r, mu, R, J2)` — Earth’s oblateness perturbation.
   - `srp_accel(r_sun, Cr, A_over_m)` — a (simplified) solar-sail push.
2. **Dynamics** (turn accelerations into state derivatives)
   - `two_body_dynamics(t, y, p)` → `[v, a_g]`
   - `j2_dynamics(t, y, p)` → `[v, a_g + a_J2]`
   - `j2_srp_dynamics(t, y, p)` → `[v, a_g + a_J2 + a_SRP]`
3. **Integrators** (advance the state)
   - `euler_step(...)` — simple, good for learning.
   - `rk4_step(...)` — higher accuracy and stability.
4. **Propagator loop**
   - `propagate(f, y0, [t0, tf], dt, params, stepper)` → arrays of times `T` and states `Y`.
5. **Diagnostics + Plotting**
   - `specific_energy(r, v, mu)`, `angular_momentum(r, v)`.
   - Beautiful **interactive 3D** orbits with a **live-size Earth** sphere.
6. **Ensemble & Ranking**
   - Run many random initial orbits and **rank the top 10** by **SMA growth** with sail ON.

Data flow:
$$ y_0 \xrightarrow{\text{integrator + dynamics}} (T, Y) \xrightarrow{\text{plots/diagnostics}} \text{Insight!} $$

### What’s marked as student work?
Every **TO IMPLEMENT** section has:
- A plain-English explanation above it.
- A docstring with **expected inputs/outputs**.
- A commented solution right below your TODOs.


## 0. Setup & constants

In [ ]:

# Plotly/ipywidgets are optional. We fall back to Matplotlib if missing.
try:
    import plotly  # noqa: F401
    _HAS_PLOTLY = True
except Exception:
    _HAS_PLOTLY = False

try:
    import ipywidgets as widgets  # noqa: F401
    _HAS_WIDGETS = True
except Exception:
    _HAS_WIDGETS = False

import math
import numpy as np
from typing import Callable, Tuple, Dict, Any

# Physical constants (SI)
MU_EARTH = 3.986004418e14        # Earth's GM [m^3/s^2]
R_EARTH  = 6378.137e3            # Earth's mean equatorial radius [m]
J2_EARTH = 1.08262668e-3         # Earth's J2 [-]
AU       = 1.495978707e11        # Astronomical Unit [m]
P_SR_1AU = 4.56e-6               # Solar radiation pressure @ 1 AU [N/m^2] (approx)

np.set_printoptions(suppress=True, precision=5)



## 1. Physics primer — state & equations

The 3D **state vector** stacks position and velocity:
$$
  y = \begin{bmatrix}x \\ y \\ z \\ v_x \\ v_y \\ v_z\end{bmatrix}
  = \begin{bmatrix}\mathbf r \\ \mathbf v\end{bmatrix}
$$

Two-body gravity:
$$
  \ddot{\mathbf r} = -\mu \frac{\mathbf r}{\lVert\mathbf r\rVert^3}
$$

State derivative function returns:
$$
  \dot y = f(t,y) = \begin{bmatrix}\mathbf v \\ \mathbf a(\mathbf r)\end{bmatrix}
$$
where $\mathbf a(\mathbf r)$ can include **J2** and **SRP** terms.



## 2. Accelerations (physics building blocks)
Below are your first **TO IMPLEMENT** cells. Read carefully, then code.



### 2.1 TO IMPLEMENT — Gravity acceleration
Return $\mathbf a_g(\mathbf r) = -\mu\,\mathbf r/\lVert\mathbf r\rVert^3$.


In [ ]:

def gravity_accel(r: np.ndarray, mu: float) -> np.ndarray:
    """
    Compute gravitational acceleration at position r from a central body.

    Parameters
    ----------
    r : np.ndarray
        Position vector [m], shape (3,).
    mu : float
        Gravitational parameter of central body [m^3/s^2].

    Returns
    -------
    np.ndarray
        Acceleration [m/s^2], shape (3,).
    """
    # TODO: a_g = -mu * r / ||r||^3
    raise NotImplementedError("Implement gravity_accel(r, mu)")
    # --- Solution ---
    # rnorm = np.linalg.norm(r)
    # return -mu * r / (rnorm**3 + 1e-30)



### 2.2 TO IMPLEMENT — J2 acceleration (Earth’s oblateness)
Use this standard approximation (good enough for outreach demos):
$$
\mathbf a_{J2} = \frac{3}{2} J_2 \frac{\mu R^2}{r^5}
\begin{bmatrix}
x\,(5z^2/r^2 - 1) \\ y\,(5z^2/r^2 - 1) \\ z\,(5z^2/r^2 - 3)
\end{bmatrix},
\quad r=\lVert\mathbf r\rVert.
$$


In [ ]:

def j2_accel(r: np.ndarray, mu: float, R: float, J2: float) -> np.ndarray:
    """
    Compute J2 perturbation acceleration for an oblate central body (Earth).

    Parameters
    ----------
    r : np.ndarray
        Position [m], shape (3,).
    mu : float
        GM [m^3/s^2].
    R : float
        Equatorial radius [m].
    J2 : float
        J2 coefficient [-].

    Returns
    -------
    np.ndarray
        J2 acceleration [m/s^2], shape (3,).
    """
    # TODO: implement the J2 formula from the description above
    raise NotImplementedError("Implement j2_accel(r, mu, R, J2)")
    # --- Solution ---
    # x, y, z = r
    # r2 = float(np.dot(r, r))
    # r1 = math.sqrt(r2) + 1e-30
    # rz2 = (z*z) / r2
    # k = 1.5 * J2 * mu * (R**2) / (r1**5)
    # ax = k * x * (5.0*rz2 - 1.0)
    # ay = k * y * (5.0*rz2 - 1.0)
    # az = k * z * (5.0*rz2 - 3.0)
    # return np.array([ax, ay, az], dtype=float)



### 2.3 TO IMPLEMENT — Solar radiation pressure (SRP)
Simple radial push from the Sun line:
$$
P(r_\odot) = P_{1\,\text{AU}}\left(\frac{1\,\text{AU}}{r_\odot}\right)^2,\quad
\mathbf a_{\text{srp}} = C_r\,\frac{A}{m}\,P(r_\odot)\,\hat{\mathbf s}.
$$
We will **exaggerate** $A/m$ and set $C_r=2$ for a dramatic ON/OFF demo.


In [ ]:

def srp_accel(r_sun: np.ndarray, Cr: float, A_over_m: float) -> np.ndarray:
    """
    Simple SRP acceleration along Sun->spacecraft direction.

    Parameters
    ----------
    r_sun : np.ndarray
        Sun->spacecraft vector [m], shape (3,).
    Cr : float
        Reflectivity coefficient (~1 to 2).
    A_over_m : float
        Area-to-mass ratio [m^2/kg] (exaggerate for demo).

    Returns
    -------
    np.ndarray
        SRP acceleration [m/s^2], shape (3,).
    """
    # TODO: implement SRP formula from the description above
    raise NotImplementedError("Implement srp_accel(r_sun, Cr, A_over_m)")
    # --- Solution ---
    # rs = np.linalg.norm(r_sun)
    # if rs < 1.0: return np.zeros(3)
    # P = P_SR_1AU * (AU/rs)**2
    # s_hat = r_sun / rs
    # return Cr * A_over_m * P * s_hat


## 3. Dynamics (turn accelerations into $\dot y$)

Dynamics functions transform physics into the mathematical form required by integrators. The **right-hand side (RHS)** of our differential equation $\dot{\mathbf{y}} = f(t, \mathbf{y})$ must return the time derivative of the state vector.

**State vector structure:**
$$\mathbf{y} = \begin{bmatrix}\mathbf{r} \\ \mathbf{v}\end{bmatrix} = \begin{bmatrix}x \\ y \\ z \\ v_x \\ v_y \\ v_z\end{bmatrix}$$

**Time derivative:**
$$\dot{\mathbf{y}} = \begin{bmatrix}\dot{\mathbf{r}} \\ \dot{\mathbf{v}}\end{bmatrix} = \begin{bmatrix}\mathbf{v} \\ \mathbf{a}_{total}\end{bmatrix}$$

**Implementation pattern:**
1. **Extract**: Split `y` into position `r = y[:3]` and velocity `v = y[3:]`
2. **Compute**: Calculate total acceleration `a_total = a_gravity + a_J2 + a_SRP + ...`
3. **Stack**: Return `[v, a_total]` using `np.hstack((v, a_total))`

**RHS for each case:**
- **Two-body**: $f = [\mathbf{v}, \mathbf{a}_{gravity}]$
- **J2**: $f = [\mathbf{v}, \mathbf{a}_{gravity} + \mathbf{a}_{J2}]$  
- **J2+SRP**: $f = [\mathbf{v}, \mathbf{a}_{gravity} + \mathbf{a}_{J2} + \mathbf{a}_{SRP}]$

In [ ]:
def two_body_dynamics(t: float, y: np.ndarray, params: Dict[str, Any]) -> np.ndarray:
    """
    Two-body dynamics.

    Parameters
    ----------
    t : float
        Time [s]. (Unused, but kept for API consistency.)
    y : np.ndarray
        State [x, y, z, vx, vy, vz], shape (6,).
    params : dict
        Must include 'mu': float.

    Returns
    -------
    np.ndarray
        dy/dt, shape (6,).
    """
    # TODO: split y into r and v, compute a_g, return [v, a_g].
    raise NotImplementedError("Implement two_body_dynamics")
    # --- Solution ---
    # r = y[:3]
    # v = y[3:]
    # a_g = gravity_accel(r, params["mu"])
    # return np.hstack((v, a_g))

In [ ]:
def j2_dynamics(t: float, y: np.ndarray, params: Dict[str, Any]) -> np.ndarray:
    """
    Gravity + J2 dynamics.

    params must include 'mu', 'R', 'J2'.
    """
    # TODO: add gravity_accel + j2_accel
    raise NotImplementedError("Implement j2_dynamics")
    # --- Solution ---
    # r = y[:3]
    # v = y[3:]
    # a = gravity_accel(r, params["mu"]) + j2_accel(r, params["mu"], params["R"], params["J2"])
    # return np.hstack((v, a))

In [ ]:
def j2_srp_dynamics(t: float, y: np.ndarray, params: Dict[str, Any]) -> np.ndarray:
    """
    Gravity + J2 + SRP dynamics.

    params must include 'mu', 'R', 'J2', 'sun_vec', 'Cr', 'A_over_m'.
    """
    # TODO: combine all three accelerations
    raise NotImplementedError("Implement j2_srp_dynamics")
    # --- Solution ---
    # r = y[:3]
    # v = y[3:]
    # a = gravity_accel(r, params["mu"]) + j2_accel(r, params["mu"], params["R"], params["J2"])
    # a += srp_accel(params["sun_vec"], params["Cr"], params["A_over_m"])
    # return np.hstack((v, a))

## 4. Integrators (advance the state)

Integrators solve $\dot{\mathbf{y}} = f(t, \mathbf{y})$ numerically by approximating the solution over small time steps $\Delta t$.

### **Explicit Euler Method**
The simplest first-order method uses a linear approximation:
$$\mathbf{y}_{n+1} = \mathbf{y}_n + \Delta t \cdot f(t_n, \mathbf{y}_n)$$

**Implementation logic:**
1. Evaluate RHS at current state: `dydt = f(t, y, params)`
2. Take linear step: `y_next = y + dt * dydt`

### **4th-Order Runge-Kutta (RK4)**
Much more accurate method using weighted average of 4 slope estimates:
$$\mathbf{y}_{n+1} = \mathbf{y}_n + \frac{\Delta t}{6}(k_1 + 2k_2 + 2k_3 + k_4)$$

Where:
- $k_1 = f(t_n, \mathbf{y}_n)$ — slope at beginning
- $k_2 = f(t_n + \frac{\Delta t}{2}, \mathbf{y}_n + \frac{\Delta t}{2}k_1)$ — slope at midpoint using $k_1$
- $k_3 = f(t_n + \frac{\Delta t}{2}, \mathbf{y}_n + \frac{\Delta t}{2}k_2)$ — slope at midpoint using $k_2$  
- $k_4 = f(t_n + \Delta t, \mathbf{y}_n + \Delta t k_3)$ — slope at end using $k_3$

**Implementation logic:**
1. Compute four slope estimates at strategic points
2. Combine with weights (1, 2, 2, 1) and scale by $\Delta t/6$
3. Add to current state for next step

In [ ]:
def euler_step(f: Callable, t: float, y: np.ndarray, dt: float, params: Dict[str, Any]) -> np.ndarray:
    """
    One explicit Euler step.

    Parameters
    ----------
    f : Callable
        Dynamics function: f(t, y, params) -> dy/dt (shape (6,)).
    t : float
        Current time [s].
    y : np.ndarray
        Current state [6].
    dt : float
        Timestep [s].
    params : dict
        Parameters passed to f.

    Returns
    -------
    np.ndarray
        Next state y_{n+1}, shape (6,).
    """
    # TODO: y_next = y + dt * f(t, y, params)
    raise NotImplementedError("Implement euler_step(...)")
    # --- Solution ---
    # return y + dt * f(t, y, params)

In [ ]:
def rk4_step(f: Callable, t: float, y: np.ndarray, dt: float, params: Dict[str, Any]) -> np.ndarray:
    """
    One classic 4th-order Runge–Kutta step.

    Returns
    -------
    np.ndarray
        Next state y_{n+1}, shape (6,).
    """
    # TODO: implement RK4 using k1, k2, k3, k4
    raise NotImplementedError("Implement rk4_step(...)")
    # --- Solution ---
    # k1 = f(t, y, params)
    # k2 = f(t + 0.5*dt, y + 0.5*dt*k1, params)
    # k3 = f(t + 0.5*dt, y + 0.5*dt*k2, params)
    # k4 = f(t + dt,       y + dt*k3,     params)
    # return y + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

## 5. Propagator (the main integration loop)

The propagator orchestrates the entire simulation by repeatedly calling your chosen integrator to march through time, storing the complete trajectory.

### **Algorithm Logic:**
```
1. Setup:
   - Calculate number of steps: N = ceil((tf - t0) / dt) + 1
   - Allocate storage arrays: T[N], Y[N×6]
   - Store initial conditions: T[0] = t0, Y[0] = y0

2. Time-stepping loop (i = 1 to N-1):
   - Advance state: y = stepper(dynamics, t, y, dt, params)
   - Advance time: t = t + dt
   - Store results: T[i] = t, Y[i] = y

3. Return complete trajectory: (T, Y)
```

### **Key Components:**
- **`f`**: Dynamics function (e.g., `j2_srp_dynamics`)
- **`stepper`**: Integration method (e.g., `rk4_step`, `euler_step`)  
- **`params`**: Physical parameters dictionary (μ, R, J2, etc.)

### **Data Flow:**
$$\text{Initial state } \mathbf{y}_0 \xrightarrow{\text{stepper + dynamics}} \mathbf{y}_1 \xrightarrow{\text{stepper + dynamics}} \mathbf{y}_2 \xrightarrow{\ldots} \mathbf{y}_N$$

The result is arrays `T` (times) and `Y` (states) ready for plotting and analysis.

In [ ]:
def propagate(f: Callable, y0: np.ndarray, tspan: Tuple[float, float], dt: float,
              params: Dict[str, Any], stepper: Callable) -> Tuple[np.ndarray, np.ndarray]:
    """
    Integrate y' = f(t, y, params) forward in time.

    Parameters
    ----------
    f : Callable
        Dynamics function -> dy/dt, shape (6,).
    y0 : np.ndarray
        Initial state [x, y, z, vx, vy, vz], shape (6,).
    tspan : (float, float)
        (t0, tf) time interval [s].
    dt : float
        Timestep [s].
    params : dict
        Parameters passed to f.
    stepper : Callable
        One-step method (e.g., euler_step or rk4_step).

    Returns
    -------
    (T, Y) : Tuple[np.ndarray, np.ndarray]
        Times shape (N,), states shape (N, 6).
    """
    # TODO: allocate arrays, loop from t0 to tf calling stepper, store results.
    raise NotImplementedError("Implement propagate(...)")
    # --- Solution ---
    # t0, tf = tspan
    # N = int(math.ceil((tf - t0) / dt)) + 1
    # T = np.zeros(N)
    # Y = np.zeros((N, 6))
    # T[0] = t0
    # Y[0] = y0.copy()
    # t = t0
    # y = y0.copy()
    # for i in range(1, N):
    #     y = stepper(f, t, y, dt, params)
    #     t = t + dt
    #     T[i] = t
    #     Y[i] = y
    # return T, Y

## 6. Diagnostics (energy, angular momentum, semi-major axis)

These functions extract orbital elements and physical quantities from the raw state vector, enabling analysis of orbit evolution.

### **Specific Orbital Energy**
The total mechanical energy per unit mass:
$$\varepsilon = \frac{1}{2}\|\mathbf{v}\|^2 - \frac{\mu}{\|\mathbf{r}\|}$$
- **Kinetic term**: $\frac{1}{2}\|\mathbf{v}\|^2$ (motion energy)
- **Potential term**: $-\frac{\mu}{\|\mathbf{r}\|}$ (gravitational potential)
- **Units**: [J/kg] or [m²/s²]

### **Specific Angular Momentum**  
Conserved quantity (in unperturbed case):
$$\mathbf{h} = \mathbf{r} \times \mathbf{v}$$
- **Magnitude**: $\|\mathbf{h}\|$ determines orbit shape
- **Direction**: $\hat{\mathbf{h}}$ defines orbital plane normal
- **Units**: [m²/s]

### **Semi-Major Axis**
Orbit size parameter from vis-viva equation:
$$a = -\frac{\mu}{2\varepsilon}$$
- **Elliptical orbits**: $\varepsilon < 0 \Rightarrow a > 0$
- **Parabolic orbits**: $\varepsilon = 0 \Rightarrow a = \infty$  
- **Hyperbolic orbits**: $\varepsilon > 0 \Rightarrow a < 0$

**Usage for ranking**: We'll compute $\Delta a = a_{final} - a_{initial}$ to measure which orbits gain the most semi-major axis (altitude) from solar sailing.

In [ ]:
def specific_energy(r: np.ndarray, v: np.ndarray, mu: float) -> float:
    """
    Specific orbital energy ε = ||v||^2/2 − mu/||r|| [J/kg].
    """
    # TODO: kinetic - potential energy
    raise NotImplementedError("Implement specific_energy")
    # --- Solution ---
    # return 0.5*np.dot(v, v) - mu / (np.linalg.norm(r) + 1e-30)

def angular_momentum(r: np.ndarray, v: np.ndarray) -> np.ndarray:
    """
    Specific angular momentum h = r × v [m^2/s].
    """
    # TODO: cross product
    raise NotImplementedError("Implement angular_momentum")
    # --- Solution ---
    # return np.cross(r, v)

def semimajor_axis_from_state(y: np.ndarray, mu: float) -> float:
    """
    Compute semi-major axis a from state vector using ε.
    Returns a in meters (could be negative for hyperbolic).
    """
    # TODO: use specific energy and vis-viva equation
    raise NotImplementedError("Implement semimajor_axis_from_state")
    # --- Solution ---
    # r = y[:3]; v = y[3:]
    # eps = specific_energy(r, v, mu)
    # return -mu / (2.0*eps + 1e-30)


## 7. Plotting — interactive 3D orbits with live-size Earth
These helpers draw trajectories and a sphere of radius **$R_{Earth}$**.
Plotly → drag/zoom in 3D. If Plotly isn't installed, we fall back to Matplotlib.


In [ ]:

def _earth_mesh(n=30):
    """Create an Earth sphere mesh (x,y,z) of live size R_EARTH."""
    u = np.linspace(0, 2*np.pi, n)
    v = np.linspace(0, np.pi, n)
    x = R_EARTH*np.outer(np.cos(u), np.sin(v))
    y = R_EARTH*np.outer(np.sin(u), np.sin(v))
    z = R_EARTH*np.outer(np.ones_like(u), np.cos(v))
    return x, y, z

def plot_3d_trajectory_list(trajectories, labels, title="3D Orbits (Live-size Earth)"):
    """
    Plot one or multiple trajectories (list of Y arrays). Uses Plotly if available.

    Parameters
    ----------
    trajectories : list[np.ndarray]
        List of state histories; each shape (N, 6).
    labels : list[str]
        Names for each trajectory.
    title : str
        Plot title.
    """
    Rs = [Y[:, :3] for Y in trajectories]
    if '_HAS_PLOTLY' in globals() and _HAS_PLOTLY:
        import plotly.graph_objects as go
        fig = go.Figure()
        for r, label in zip(Rs, labels):
            fig.add_trace(go.Scatter3d(x=r[:,0], y=r[:,1], z=r[:,2], mode='lines', name=label))
        # Earth sphere
        X, Y, Z = _earth_mesh(45)
        fig.add_trace(go.Surface(x=X, y=Y, z=Z, opacity=0.5, showscale=False, name="Earth"))
        fig.update_layout(title=title,
                          scene=dict(xaxis_title='x [m]',
                                     yaxis_title='y [m]',
                                     zaxis_title='z [m]',
                                     aspectmode='data'),
                          showlegend=True)
        fig.show()
    else:
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        for r, label in zip(Rs, labels):
            ax.plot(r[:,0], r[:,1], r[:,2], label=label)
        # Earth wireframe
        X, Y, Z = _earth_mesh(25)
        ax.plot_wireframe(X, Y, Z, rstride=2, cstride=2, alpha=0.3)
        ax.set_title(title)
        ax.set_xlabel("x [m]"); ax.set_ylabel("y [m]"); ax.set_zlabel("z [m]")
        ax.set_box_aspect([1,1,1])
        ax.legend()
        plt.show()



## 8. Initial conditions (inclined orbits for visible RAAN drift)
We’ll start with an **inclined circular orbit** so J2 causes RAAN to drift.


In [ ]:

def Rx(theta: float) -> np.ndarray:
    c, s = math.cos(theta), math.sin(theta)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]], dtype=float)

def Rz(theta: float) -> np.ndarray:
    c, s = math.cos(theta), math.sin(theta)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]], dtype=float)

def circular_orbit_in_plane(mu: float, radius: float) -> np.ndarray:
    """Base equatorial circular state on +x with velocity +y."""
    v = math.sqrt(mu / radius)
    r0 = np.array([radius, 0, 0], dtype=float)
    v0 = np.array([0, v, 0], dtype=float)
    return np.hstack((r0, v0))

def inclined_circular_state(mu: float, radius: float, inc_deg: float = 55.0, raan_deg: float = 30.0) -> np.ndarray:
    """
    Create an inclined circular orbit by rotating a base equatorial state.

    Parameters
    ----------
    mu : float
        GM [m^3/s^2].
    radius : float
        Orbital radius from center [m].
    inc_deg : float
        Inclination in degrees.
    raan_deg : float
        RAAN (right ascension of ascending node) in degrees.

    Returns
    -------
    np.ndarray
        State [x, y, z, vx, vy, vz], shape (6,).
    """
    y0 = circular_orbit_in_plane(mu, radius)
    inc = math.radians(inc_deg)
    raan = math.radians(raan_deg)
    R = Rz(raan) @ Rx(inc)
    r = R @ y0[:3]
    v = R @ y0[3:]
    return np.hstack((r, v))

def random_inclined_circular(mu: float, radius: float, inc_range=(30.0, 75.0)) -> np.ndarray:
    """Random RAAN, inclination, and true anomaly for ensemble tests."""
    base = circular_orbit_in_plane(mu, radius)
    inc = math.radians(np.random.uniform(*inc_range))
    raan = np.random.uniform(0, 2*np.pi)
    nu = np.random.uniform(0, 2*np.pi)
    R = Rz(raan) @ Rx(inc) @ Rz(nu)
    return np.hstack((R @ base[:3], R @ base[3:]))



## 9. Interactive single-orbit demo (J2 & SRP)
- Use an **inclined** circular orbit to visualize **RAAN drift** under J2.
- Toggle **sail ON/OFF** with exaggerated SRP to see a clear effect.


In [ ]:
def single_demo():
    """
    Interactive VS Code-friendly demo. Uses ipywidgets if available, otherwise just runs once.
    """
    def _run(inc_deg=55.0, raan_deg=30.0, altitude_km=600.0,
             duration_hr=3.0, dt=10.0, integrator="RK4",
             j2_multiplier=1.0, sail_on=True, A_over_m=10.0, Cr=2.0):
        radius = R_EARTH + altitude_km*1e3
        y0 = inclined_circular_state(MU_EARTH, radius, inc_deg=inc_deg, raan_deg=raan_deg)
        step = rk4_step if integrator.upper()=="RK4" else euler_step
        tspan = (0.0, duration_hr*3600.0)

        # J2 params
        J2 = J2_EARTH * j2_multiplier

        # Dynamics & params
        if sail_on:
            sun_dir = np.array([1.0, 0.2, 0.1]); sun_dir = sun_dir/np.linalg.norm(sun_dir)
            params = {"mu": MU_EARTH, "R": R_EARTH, "J2": J2,
                      "sun_vec": AU*sun_dir, "Cr": Cr, "A_over_m": A_over_m}
            f = j2_srp_dynamics
            label = f"J2({j2_multiplier:.1f}×) + SRP ON"
        else:
            params = {"mu": MU_EARTH, "R": R_EARTH, "J2": J2}
            f = j2_dynamics
            label = f"J2({j2_multiplier:.1f}×), SRP OFF"

        T, Y = propagate(f, y0, tspan, dt, params, step)
        plot_3d_trajectory_list([Y], [label], title="Inclined orbit: RAAN drift + SRP toggle")

    has_widgets = globals().get('_HAS_WIDGETS', False)

    if has_widgets:
        import ipywidgets as widgets
        from ipywidgets import interact, FloatSlider, Checkbox, Dropdown, IntSlider

        interact(_run,
                 inc_deg=FloatSlider(value=55.0, min=0.0, max=90.0, step=1.0, description="inc [deg]"),
                 raan_deg=FloatSlider(value=30.0, min=0.0, max=360.0, step=5.0, description="RAAN [deg]"),
                 altitude_km=FloatSlider(value=600.0, min=200.0, max=2000.0, step=50.0, description="alt [km]"),
                 duration_hr=FloatSlider(value=3.0, min=0.5, max=12.0, step=0.5, description="T [hr]"),
                 dt=FloatSlider(value=10.0, min=1.0, max=60.0, step=1.0, description="dt [s]"),
                 integrator=Dropdown(options=["RK4","Euler"], value="RK4", description="Integrator"),
                 j2_multiplier=FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description="J2 ×"),
                 sail_on=Checkbox(value=True, description="Sail ON"),
                 A_over_m=FloatSlider(value=10.0, min=0.0, max=30.0, step=1.0, description="A/m"),
                 Cr=FloatSlider(value=2.0, min=0.5, max=3.0, step=0.1, description="Cr"))
    else:
        # Non-widget fallback: run once with defaults (no recursion)
        _run()

# Auto-run demo when all functions are implemented
# single_demo()


## 10. Ensemble with ranking (top 10 SMA growth)
We create **N random inclined circular** orbits, run **J2 only** vs **J2+SRP (exaggerated)**, then **rank** the SRP-ON cases by **SMA growth**:
$$
  \Delta a = a_{\text{final}} - a_{\text{initial}}.
$$


In [ ]:
def run_ensemble(N: int = 10, duration_hr: float = 3.0, dt: float = 10.0,
                 altitude_km: float = 600.0, inc_range=(30.0, 75.0),
                 A_over_m: float = 10.0, Cr: float = 2.0,
                 j2_multiplier: float = 1.0):
    """
    Simulate an ensemble with SRP OFF (J2 only) and SRP ON (J2+SRP).
    Returns:
      results_off: list of dicts with keys {'Y', 'a0', 'aF', 'dA', 'label'}
      results_on:  list of dicts with the same keys
    """
    radius = R_EARTH + altitude_km*1e3
    tspan = (0.0, duration_hr*3600.0)
    step = rk4_step
    J2 = J2_EARTH * j2_multiplier
    sun_dir = np.array([1.0, 0.2, 0.1]); sun_dir = sun_dir/np.linalg.norm(sun_dir)

    results_off = []
    results_on = []
    for i in range(N):
        y0 = random_inclined_circular(MU_EARTH, radius, inc_range=inc_range)
        a0 = semimajor_axis_from_state(y0, MU_EARTH)

        # OFF
        params_off = {"mu": MU_EARTH, "R": R_EARTH, "J2": J2}
        T0, Y0 = propagate(j2_dynamics, y0, tspan, dt, params_off, step)
        aF0 = semimajor_axis_from_state(Y0[-1], MU_EARTH)
        results_off.append({"Y": Y0, "a0": a0, "aF": aF0, "dA": aF0-a0, "label": f"OFF #{i+1}"})

        # ON (exaggerated sail)
        params_on = {"mu": MU_EARTH, "R": R_EARTH, "J2": J2,
                     "sun_vec": AU*sun_dir, "Cr": Cr, "A_over_m": A_over_m}
        T1, Y1 = propagate(j2_srp_dynamics, y0, tspan, dt, params_on, step)
        aF1 = semimajor_axis_from_state(Y1[-1], MU_EARTH)
        results_on.append({"Y": Y1, "a0": a0, "aF": aF1, "dA": aF1-a0, "label": f"ON  #{i+1}"})

    # Rank top 10 by Δa for SRP ON
    results_on_sorted = sorted(results_on, key=lambda d: d["dA"], reverse=True)[:10]
    return results_off, results_on_sorted

# Demo run + plots (uncomment after implementing all functions)
# res_off, res_on_top = run_ensemble(N=20, duration_hr=3.0, dt=10.0, altitude_km=600.0)
# plot_3d_trajectory_list([d["Y"] for d in res_off[:5]], [d["label"] for d in res_off[:5]],
#                         title="Ensemble — SRP OFF (subset, live-size Earth)")
# plot_3d_trajectory_list([d["Y"] for d in res_on_top], [d["label"] + f" (Δa={d['dA']:.0f} m)" for d in res_on_top],
#                         title="Top 10 by SMA growth — SRP ON (exaggerated)")


## 11. Stretch goals
- Switch the sail **ON for the first half** and **OFF for the second half** of each propagation.
- Plot **energy drift** for Euler vs RK4 at different `dt`.
- Add a **ground track** view (lat/lon) for one case.

---

## 12. Wrap-up
You built a fully interactive 3D propagator with:
- Live-size Earth
- J2 with **inclined** orbits to make **RAAN drift** obvious
- Exaggerated **SRP ON/OFF** control
- An **ensemble** with ranking by **SMA growth**

Everything is written with basic, readable Python and friendly docstrings — perfect for VS Code Jupyter.
